In [10]:
from bs4 import BeautifulSoup
import csv
import pandas as pd
import requests
import smtplib
import time
import datetime

In [11]:
URL = 'https://www.amazon.com/Data-Analyst-Definition-Analysis-Distressed/dp/B0C1THH2KF/ref=sr_1_2?dib=eyJ2IjoiMSJ9.H_xD2LC8KXJb_HUr4wTcfXoE8KxzHlWEFJD26DN4qCY9Xg-IzMGnys93_FcapDMZJvHVPbNZwhcKR7SGVAfXCarX_InBqeDIp22O87EuR53FkEuWiNJrA-Mn_-i3kbS3DJLO99gkyMsNh2uY0_e8Vd_TQHOMVQoWOMf4iBBLC2NDAvBudaCs-ipfuMY-mLaNQXpA7ulEsPjXKP5hh3q052pL6bTfx5bbcvsbTtilLev6L1pnwYgkO6xUcfei9GjBxeCRt6k4bwp11V2rwqfdlv9RjVk9eSw8UOUh0m0AmPk.dQ6DqlUpON-f4dUlc8My-Fzo1j1qqDjDSbiAF1_YQzM&dib_tag=se&keywords=data%2Banalyst%2Btshirt&qid=1724913942&sr=8-2'
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:129.0) Gecko/20100101 Firefox/129.0",
    "Accept-Encoding":"gzip, deflate, br, zstd",
    "Accept":"html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/png,image/svg+xml,*/*;q=0.8",
    "DNT":"1",
    "Connection":"close",
    "Upgrade-Insecure-Requests":"1"
    }

In [12]:
page = requests.get(URL, headers=headers)

soup1 = BeautifulSoup(page.content, "html.parser")
soup1 = BeautifulSoup(soup1.prettify(), "html.parser")

title = soup1.find(id='productTitle').get_text().strip()
print(title)
price = soup1.find('span', class_='a-offscreen').get_text().strip()
price = price[1:]
print(price)
today = datetime.date.today()
print(today)

Data Analyst Definition Data Analysis Distressed T-Shirt
17.99
2024-08-29


INSERTING INTO CSV

In [13]:
header = ['Title', 'Price', 'Date']
data = [title,price,today]
with open('AmazonScraperDATASET.csv','w', newline='', encoding='UTF8') as f:
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerow(data)

CREATING PANDAS DF

In [14]:
df = pd.read_csv('AmazonScraperDATASET.csv')

print(df)

                                               Title  Price        Date
0  Data Analyst Definition Data Analysis Distress...  17.99  2024-08-29


In [15]:
def check_price():
    URL = 'https://www.amazon.com/Data-Analyst-Definition-Analysis-Distressed/dp/B0C1THH2KF/ref=sr_1_2?dib=eyJ2IjoiMSJ9.H_xD2LC8KXJb_HUr4wTcfXoE8KxzHlWEFJD26DN4qCY9Xg-IzMGnys93_FcapDMZJvHVPbNZwhcKR7SGVAfXCarX_InBqeDIp22O87EuR53FkEuWiNJrA-Mn_-i3kbS3DJLO99gkyMsNh2uY0_e8Vd_TQHOMVQoWOMf4iBBLC2NDAvBudaCs-ipfuMY-mLaNQXpA7ulEsPjXKP5hh3q052pL6bTfx5bbcvsbTtilLev6L1pnwYgkO6xUcfei9GjBxeCRt6k4bwp11V2rwqfdlv9RjVk9eSw8UOUh0m0AmPk.dQ6DqlUpON-f4dUlc8My-Fzo1j1qqDjDSbiAF1_YQzM&dib_tag=se&keywords=data%2Banalyst%2Btshirt&qid=1724913942&sr=8-2'
    headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:129.0) Gecko/20100101 Firefox/129.0",
    "Accept-Encoding":"gzip, deflate, br, zstd",
    "Accept":"html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/png,image/svg+xml,*/*;q=0.8",
    "DNT":"1",
    "Connection":"close",
    "Upgrade-Insecure-Requests":"1"
    }
    page = requests.get(URL, headers=headers)

    soup1 = BeautifulSoup(page.content, "html.parser")
    soup1 = BeautifulSoup(soup1.prettify(), "html.parser")

    title = soup1.find(id='productTitle').get_text().strip()

    price = soup1.find('span', class_='a-offscreen').get_text().strip()
    price = price[1:]

    today = datetime.date.today()

    data = [title, price, today]

    with open('AmazonScraperDATASET.csv', 'a+', newline='', encoding='UTF8') as f:
        writer = csv.writer(f)
        writer.writerow(data)
    
    return price
 

SEND MAIL EVENT FUNCTION

In [16]:
def send_mail():
    try:

        server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
        server.ehlo()
        
        server.login('yourmail@gmail.com', 'yourpassword')
        
        subject = "The Shirt you want is below $15! Now is your chance to buy!"
        body = f"Link: {URL}"
        msg = f"Subject: {subject}\n\n{body}"
        
        server.sendmail('yourmail@gmail.com', 'recipient@example.com', msg)
        
        server.quit()
        
        print("Email sent successfully.")
    except Exception as e:
        print(f"Failed to send email: {e}")

GETTING PRICE TO COMPARE WITH TRIGGER PRICE

In [8]:
def get_price(URL, headers):

    page = requests.get(URL, headers=headers)
    soup = BeautifulSoup(page.content, "html.parser")
    

    price_span = soup.find('span', class_='a-offscreen')
    if price_span:
        price_text = price_span.get_text().strip()
        price = price_text[1:]
        return float(price.replace(',', ''))

SCRAPING MAIN PROCESS

In [13]:
DAY = 86400
while(True):
    check_price()

    price = get_price(URL, headers)

    if get_price(URL, headers)>15:
        send_mail()

    time.sleep(DAY) #cheking price once a day